In [14]:
import matplotlib.pyplot as plt
from sksurv.datasets import load_whas500
from sksurv.ensemble import RandomSurvivalForest
from sklearn.model_selection import train_test_split
from lifelines import KaplanMeierFitter
import numpy as np
import itertools
import random

In [2]:
X, y = load_whas500()
X = X.astype(float)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [3]:
rsf = RandomSurvivalForest(random_state=42).fit(X_train, y_train)

In [4]:
X_test.iloc[:1,:]

,afb,age,av3,bmi,chf,cvd,diasbp,gender,hr,los,miord,mitype,sho,sysbp
361,0.0,51.0,0.0,40.03658,0.0,1.0,28.0,0.0,86.0,7.0,0.0,1.0,0.0,202.0


In [5]:
print(rsf.decision_path(X_test.iloc[:1,:]))

(<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 728 stored elements and shape (1, 10136)>, array([    0,    97,   192,   287,   388,   481,   580,   687,   794,
         891,  1004,  1101,  1202,  1299,  1396,  1511,  1612,  1703,
        1812,  1909,  2014,  2123,  2228,  2333,  2432,  2539,  2646,
        2751,  2870,  2977,  3078,  3165,  3268,  3363,  3468,  3573,
        3680,  3785,  3894,  3989,  4088,  4199,  4304,  4409,  4512,
        4613,  4718,  4815,  4910,  5025,  5130,  5229,  5340,  5441,
        5554,  5659,  5758,  5853,  5930,  6033,  6128,  6221,  6330,
        6441,  6546,  6649,  6746,  6845,  6950,  7055,  7142,  7247,
        7350,  7459,  7558,  7663,  7760,  7859,  7980,  8083,  8164,
        8257,  8354,  8441,  8546,  8647,  8744,  8853,  8958,  9075,
        9176,  9271,  9372,  9453,  9550,  9655,  9754,  9851,  9944,
       10049, 10136]))


In [6]:
tree_index = 0  # On choisit le premier arbre
tree = rsf.estimators_[tree_index].tree_

In [7]:
for node in range(78,81):
    left_child = tree.children_left[node]
    right_child = tree.children_right[node]
    feature_index = tree.feature[node]
    threshold = tree.threshold[node]
    
    print(f"Nœud {node}:")
    print(f"  Feature utilisée: {X.columns[feature_index] if feature_index != -2 else 'Leaf'}")
    print(f"  Seuil: {threshold:.3f}")
    print(f"  Fils gauche: {left_child}, Fils droit: {right_child}")
    print("-" * 30)

Nœud 78:
  Feature utilisée: sho
  Seuil: 0.500
  Fils gauche: 79, Fils droit: 94
------------------------------
Nœud 79:
  Feature utilisée: age
  Seuil: 80.500
  Fils gauche: 80, Fils droit: 91
------------------------------
Nœud 80:
  Feature utilisée: cvd
  Seuil: 0.500
  Fils gauche: 81, Fils droit: 84
------------------------------


In [8]:
def same_path(X, obs, rsf, tree_index, depth=0):
    #X entrainement, obs a etudier, rsf mdoele, tree index, l'arbre
    nodes=rsf.estimators_[tree_index].decision_path(obs).indices
    X = X.reset_index(drop=True)
    for node in nodes[depth:-1]:
        var=tree.feature[node]
        seuil=tree.threshold[node]
        X = X[X.iloc[:, var] >= seuil] if obs.iloc[0,var]>= seuil else X[X.iloc[:, var] < seuil]
    return X

same_path=same_path(X_train, X_test.iloc[:1,:], rsf, 0)


/home/onyxia/.local/lib/python3.10/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SurvivalTree was fitted without feature names
  warnings.warn(


In [9]:
def surv_est(same_path, y_train, times):
    y_train_list = np.array(y_train.tolist())[same_path.index]
    KMF= KaplanMeierFitter()
    KMF.fit(y_train_list[:, 1],y_train_list[:, 0])
    return(KMF.predict(times))

surv_est(same_path, y_train, list(range(5,100,5)))

5     1.000000
10    1.000000
15    1.000000
20    1.000000
25    1.000000
30    1.000000
35    1.000000
40    1.000000
45    1.000000
50    0.970588
55    0.970588
60    0.970588
65    0.970588
70    0.970588
75    0.970588
80    0.970588
85    0.970588
90    0.970588
95    0.970588
Name: KM_estimate, dtype: float64

In [ ]:
def rpermut(n):
    random_permutations = []

    for _ in range(n):
        shuffled = list(X.columns)  # Copier la liste
        random.shuffle(shuffled)  # Mélanger aléatoirement
        random_permutations.append(tuple(shuffled))  # Ajouter en tuple (comme permutations)

    return(random_permutations)

In [48]:
permutation=rpermut(1)
variables=[]
for variable in permutation[0]:
    variables.append(variable)
    node=0:
    while 
    

['av3']
['av3', 'chf']
['av3', 'chf', 'hr']
['av3', 'chf', 'hr', 'miord']
['av3', 'chf', 'hr', 'miord', 'diasbp']
['av3', 'chf', 'hr', 'miord', 'diasbp', 'los']
['av3', 'chf', 'hr', 'miord', 'diasbp', 'los', 'afb']
['av3', 'chf', 'hr', 'miord', 'diasbp', 'los', 'afb', 'gender']
['av3', 'chf', 'hr', 'miord', 'diasbp', 'los', 'afb', 'gender', 'bmi']
['av3', 'chf', 'hr', 'miord', 'diasbp', 'los', 'afb', 'gender', 'bmi', 'mitype']
['av3', 'chf', 'hr', 'miord', 'diasbp', 'los', 'afb', 'gender', 'bmi', 'mitype', 'age']
['av3', 'chf', 'hr', 'miord', 'diasbp', 'los', 'afb', 'gender', 'bmi', 'mitype', 'age', 'sysbp']
['av3', 'chf', 'hr', 'miord', 'diasbp', 'los', 'afb', 'gender', 'bmi', 'mitype', 'age', 'sysbp', 'cvd']
['av3', 'chf', 'hr', 'miord', 'diasbp', 'los', 'afb', 'gender', 'bmi', 'mitype', 'age', 'sysbp', 'cvd', 'sho']


In [36]:
class parasite:
    def __init__(self, number, left=None, right=None, future_features=None, feature=None, value=None, weight=None, path=None):
        self.number=number
        self.left=left
        self.right=right
        self.future_features=future_features
        self.feature=feature
        self.value=value
        self.weight=weight
        self.path=None
    
    def copy_tree(self, tree):
        if tree.feature[number] == -2:
            return (parasite(number=number, value=))



'age'

In [49]:
class Arbre:
    def __init__(self, valeur, gauche=None, droite=None):
        """Initialise un arbre avec une valeur et éventuellement deux sous-arbres."""
        self.valeur = valeur
        self.gauche = gauche
        self.droite = droite

    def afficher(self, niveau=0):
        """Affiche l'arbre de manière structurée."""
        if self.droite:
            self.droite.afficher(niveau + 1)
        print("    " * niveau + str(self.valeur))
        if self.gauche:
            self.gauche.afficher(niveau + 1)

# Exemple d'utilisation
arbre = Arbre(10, Arbre(5, Arbre(2), Arbre(7)), Arbre(15, None, Arbre(20)))
arbre.afficher()

        20
    15
10
        7
    5
        2


In [57]:
leaf_indices = rsf.apply(X)
leaf_indices[:,0]

array([53, 87, 82, 87, 87,  1, 87, 89, 65, 14, 83, 90, 14, 27, 19,  8, 25,
       83, 83, 86, 11, 72, 72, 83, 18, 14, 36, 44, 35, 76, 52, 23, 87, 93,
       53, 87, 37, 52, 73, 18, 90, 73, 30,  1, 73, 18, 90, 14, 65, 14, 48,
       11, 55, 44, 95, 86, 17, 17, 52, 87, 60, 86, 90, 93, 90, 95, 48, 82,
       87,  1, 73, 12, 83, 30, 92, 87, 14, 82, 11, 53, 86, 82, 43, 87, 48,
       47, 14, 40, 50, 44, 77, 44, 30, 47, 82, 27, 27, 95, 37, 14, 61, 73,
       28, 73, 93, 83, 11, 44, 19, 14, 83, 69, 93, 87, 56, 12, 73, 83, 11,
       90, 12, 92, 14, 87, 69, 87, 83, 14, 19, 44, 68, 83, 65, 11, 43, 23,
       71, 82, 87, 65, 83, 83, 92, 90, 87, 30, 12, 48, 83, 69, 44, 14, 20,
       87, 30, 28, 61, 95, 44, 96, 30, 14, 37, 43, 96, 25, 14, 14, 90, 93,
       83, 48, 25, 19,  1,  8, 93, 44, 72, 77, 47, 11, 23, 53, 83, 87, 14,
       76, 86, 87, 87, 83, 14, 87, 82, 47,  1, 86, 60, 90, 83, 23, 60, 87,
       14, 44, 40, 83, 47, 35, 60, 87, 83, 40, 50, 65, 87, 43, 44, 83, 52,
       76, 44, 35, 52, 90